In [ ]:
from train import train
from utils import Config

class CFG(Config):
    seed = 42
    cv_fold = 5
    base_model = 'efficientnet_b0'   # resnet18/34/50, efficientnet_b0/b1/b2/b3/b4, efficientnet_v2_s, convnext_tiny, swin_t
    batch_size = 32
    epochs = 3
    base_lr = 1e-3
    affine_degrees = 0
    affine_translate = None
    affine_scale = None
    dataloader_num_workers = 8
    scheduler_step_size = 2
    optimizer = 'AdamW'
    scheduler = 'StepLR'
    loss = 'KLDivLoss'
    lr_gamma = 0.1
    sgd_momentum = 0.9
    color_jitter_args = dict(p=0.0, brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)
    random_erasing_p = 0
    freeze_epochs = 0
    spec_trial_selection = 'all' # 'all', 'first', 'mean_offset'
    eeg_trial_selection = 'first' # 'all', 'first', 'mean_offset'
    spec_random_trial_num = 1
    eeg_random_trial_num = 1
    TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
    coarse_dropout_args = dict(p=0.5, max_holes=8, max_height=128, max_width=128)
    data_type = 'eeg_tf'  # 'spec', 'eeg_tf', 'spec+eeg_tf
    eeg_tf_data = 'eeg_tf_data_globalnorm'
    pretrained = True

    if data_type == 'spec':
        in_channels = 1
    elif data_type == 'eeg_tf':
        in_channels = 1
    elif data_type == 'spec+eeg_tf':
        in_channels = 2

tags=['torch', 'cv', 'best_epoch']
notes = ''
train_final_model = False
use_wandb = True
one_fold = False


for scheduler in ['StepLR']:
    CFG.scheduler = scheduler
    for epochs in [3]:
        CFG.epochs = epochs
        for eeg_tf_data in ['eeg_tf_data_globalnorm']:
            CFG.eeg_tf_data = eeg_tf_data
            for base_model in ['efficientnet_b0']:
                CFG.base_model = base_model
                for coarse_dropout_args in [dict(p=0.0, max_holes=8, max_height=128, max_width=128), dict(p=0.5, max_holes=4, max_height=128, max_width=128), dict(p=1.0, max_holes=4, max_height=128, max_width=128)]:
                    CFG.coarse_dropout_args = coarse_dropout_args
                    for batch_size in [32]:
                        CFG.batch_size = batch_size
                        for eeg_trial_selection in ['first']:
                            CFG.eeg_trial_selection = eeg_trial_selection
                            for eeg_random_trial_num in [1]:
                                CFG.eeg_random_trial_num = eeg_random_trial_num
                                for data_type in ['eeg_tf']:
                                    CFG.data_type = data_type
                                    if data_type == 'spec':
                                        CFG.in_channels = 1
                                    elif data_type == 'eeg_tf':
                                        CFG.in_channels = 1
                                    elif data_type == 'spec+eeg_tf':
                                        CFG.in_channels = 2
                                    print(f'##########################\nTraining with \
                                        epochs {epochs}, \
                                        scheduler {scheduler}, \
                                        base_model: {base_model}, \
                                        eeg_tf_fname {eeg_tf_data}, \
                                        coarse_dropout_args: {coarse_dropout_args} \
                                        data_type {data_type}, \
                                        \n##########################\n')
                                    train(CFG, tags=tags, notes=notes, train_final_model=train_final_model, use_wandb=use_wandb, one_fold=one_fold)